In [1]:
import torch
import random

import torch.nn.functional as F
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf
import torch.nn.functional as F
from botorch.acquisition import ExpectedImprovement



m:\workspace\bo\botorch_contrast\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
BATCHSIZE = 20
TARGET = [0.34, 0.2]


#r = torch.tensor([[random.uniform(0, 1), random.uniform(0, 1)]], dtype=torch.double)  
def dif(tensor, target):
  target = torch.tensor([target], dtype=torch.double)
  r = target - tensor 
  r = torch.abs(r)
  r = torch.sum(r)
  #r = torch.clamp(r,0,1)
  return 1-torch.tensor([[r]], dtype=torch.double)
  


def getSample():
  return torch.tensor([[random.uniform(0, 1), random.uniform(0, 1)]], dtype=torch.double)  

X = torch.tensor([], dtype=torch.double)
Y = torch.tensor([], dtype=torch.double)
# Create a Standardize transform using the data

for i in range(BATCHSIZE):
  x = getSample()
  X = torch.cat((X, x), dim=0)
  y = dif(x, TARGET)
  Y = torch.cat((Y, y), dim=0)
  
  
Y =  F.normalize(Y, dim=None, p=2)

print(Y)


list(zip(X, Y))


tensor([[ 0.0487],
        [ 0.4119],
        [ 0.3728],
        [ 0.1304],
        [ 0.2631],
        [ 0.1876],
        [ 0.1470],
        [ 0.2308],
        [ 0.0564],
        [ 0.2285],
        [ 0.2560],
        [ 0.1747],
        [ 0.2208],
        [ 0.1720],
        [-0.0207],
        [ 0.1784],
        [ 0.0801],
        [ 0.3041],
        [ 0.3625],
        [ 0.0172]], dtype=torch.float64)


[(tensor([0.8211, 0.6132], dtype=torch.float64),
  tensor([0.0487], dtype=torch.float64)),
 (tensor([0.3405, 0.3054], dtype=torch.float64),
  tensor([0.4119], dtype=torch.float64)),
 (tensor([0.4217, 0.0910], dtype=torch.float64),
  tensor([0.3728], dtype=torch.float64)),
 (tensor([0.0035, 0.5804], dtype=torch.float64),
  tensor([0.1304], dtype=torch.float64)),
 (tensor([0.5496, 0.4194], dtype=torch.float64),
  tensor([0.2631], dtype=torch.float64)),
 (tensor([0.2760, 0.7287], dtype=torch.float64),
  tensor([0.1876], dtype=torch.float64)),
 (tensor([0.7935, 0.4275], dtype=torch.float64),
  tensor([0.1470], dtype=torch.float64)),
 (tensor([0.2313, 0.5902], dtype=torch.float64),
  tensor([0.2308], dtype=torch.float64)),
 (tensor([0.5715, 0.8462], dtype=torch.float64),
  tensor([0.0564], dtype=torch.float64)),
 (tensor([0.6866, 0.3573], dtype=torch.float64),
  tensor([0.2285], dtype=torch.float64)),
 (tensor([0.7461, 0.1618], dtype=torch.float64),
  tensor([0.2560], dtype=torch.float64)),

In [13]:
train_Y = standardize(Y)
#train_Y = Y
gp = SingleTaskGP(X, train_Y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)
bounds = torch.stack([torch.zeros(2), torch.ones(2)])

In [7]:

# Define the Expected Improvement (EI) acquisition function
ei = ExpectedImprovement(gp, best_f=torch.max(Y))

# 'best_observed_value' is the best observed value from your optimization process so far
bounds = torch.stack([torch.zeros(2), torch.ones(2)])

# Optimize the EI acquisition function to find the next candidate point
candidate, acq_value = optimize_acqf(
    ei, bounds=bounds, q=1, num_restarts=20, raw_samples=150,
)

print(candidate)

tensor([[0.3920, 0.1869]])
